
## Installs


In [4]:

!pip install xgboost
!pip install onnxmltools
!pip install onnxruntime


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
   - -------------------------------------- 2.4/72.0 MB 15.0 MB/s eta 0:00:05
   --- ------------------------------------ 6.3/72.0 MB 16.8 MB/s eta 0:00:04
   ----- ---------------------------------- 10.0/72.0 MB 17.3 MB/s eta 0:00:04
   ------- -------------------------------- 13.6/72.0 MB 17.5 MB/s eta 0:00:04
   --------- ------------------------------ 17.6/72.0 MB 17.6 MB/s eta 0:00:04
   ----------- ---------------------------- 21.5/72.0 MB 17.7 MB/s eta 0:00:03
   -------------- ------------------------- 25.7/72.0 MB 17.9 MB/s eta 0:00:03
   ---------------- ----------------------- 29.4/72.0 MB 18.1 MB/s eta 0:00:03
   ------------------ --------------------- 33.3/72.0 MB 18.1 MB/s eta 0:00:03
   -------------------- ------------------- 37.2/72.0 MB 18.1 MB/s eta 0:

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.18.0 requires ml-dtypes<0.5.0,>=0.4.0, but you have ml-dtypes 0.5.4 which is incompatible.


Defaulting to user installation because normal site-packages is not writeable
  Using cached onnxruntime-1.23.2-cp312-cp312-win_amd64.whl.metadata (5.3 kB)
  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl.metadata (12 kB)
  Using cached humanfriendly-10.0-py2.py3-none-any.whl.metadata (9.2 kB)
  Using cached pyreadline3-3.5.4-py3-none-any.whl.metadata (4.7 kB)
Using cached onnxruntime-1.23.2-cp312-cp312-win_amd64.whl (13.5 MB)
Using cached coloredlogs-15.0.1-py2.py3-none-any.whl (46 kB)
Using cached humanfriendly-10.0-py2.py3-none-any.whl (86 kB)
Using cached pyreadline3-3.5.4-py3-none-any.whl (83 kB)

   ---------------------------------------- 0/4 [pyreadline3]
   ---------------------------------------- 0/4 [pyreadline3]
   ---------------------------------------- 0/4 [pyreadline3]
   ---------------------------------------- 0/4 [pyreadline3]
   ---------------------------------------- 0/4 [pyreadline3]
   ---------------------------------------- 0/4 [pyreadline3]
   ---------


## Imports


In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [7]:

import torch
import numpy as np
import pandas as pd
import sklearn
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
import math
import seaborn as sns
import random
import functorch


import torch.optim as optim


from numpy.random import normal
from scipy.stats import norm
import scipy.stats as stats
from numpy import hstack
from numpy import vstack
from numpy import exp
from sklearn.neighbors import KernelDensity

##from typing import Optional

from torch import Tensor


from torch.utils.data import TensorDataset, DataLoader

from mlxtend.plotting import heatmap

## coefficient of determination (R**2)
from sklearn.metrics import r2_score

np.set_printoptions(suppress=True)
torch.set_printoptions(sci_mode=False)

In [9]:

import xgboost as xgb
from xgboost import XGBClassifier, XGBRegressor, DMatrix, train as train_xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

import onnxmltools
from onnxmltools.convert.xgboost.operator_converters.XGBoost import (
    convert_xgboost)
from onnxmltools.convert import convert_xgboost as convert_xgboost_booster


from zipfile import ZipFile
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import convert_sklearn, to_onnx, update_registered_converter
from skl2onnx.common.shape_calculator import (
    calculate_linear_classifier_output_shapes,
    calculate_linear_regressor_output_shapes)


# Import the ONNX runtime environment
import onnxruntime as rt

## Preprocessing

In [12]:
df = pd.read_csv("movie_metadata.csv")
df.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


In [13]:
# ------------------------
# 1. Drop columns we don't need
# ------------------------
drop_cols = [
    'movie_title', 'plot_keywords', 'movie_imdb_link', 'color'
]
df = df.drop(columns=drop_cols)

# ------------------------
# 2. Fill missing values
# ------------------------
# Numeric columns
numeric_cols = [
    'budget', 'title_year', 'duration', 'num_critic_for_reviews',
    'num_user_for_reviews', 'facenumber_in_poster'
]
for col in numeric_cols:
    df[col] = df[col].fillna(df[col].median())

# Categorical columns
cat_cols = ['genres', 'director_name', 'actor_1_name', 'actor_2_name',
            'actor_3_name', 'content_rating', 'country', 'language']
for col in cat_cols:
    df[col] = df[col].fillna('Unknown')

# ------------------------
# 3. Process multi-label 'genres'
# ------------------------
all_genres = set()
df['genres'] = df['genres'].apply(lambda x: x.split('|'))
for genre_list in df['genres']:
    all_genres.update(genre_list)
all_genres = sorted(list(all_genres))

# Multi-hot encoding
for genre in all_genres:
    df[f'genre_{genre}'] = df['genres'].apply(lambda x: 1 if genre in x else 0)
df = df.drop(columns=['genres'])

# ------------------------
# 4. One-hot encode small categorical features
# ------------------------
one_hot_cols = ['content_rating', 'country', 'language']
df = pd.get_dummies(df, columns=one_hot_cols, drop_first=True)

# ------------------------
# 5. Encode top directors and actors (top-10 only)
# ------------------------
top_directors = df['director_name'].value_counts().nlargest(10).index
for director in top_directors:
    df[f'director_{director}'] = df['director_name'].apply(lambda x: 1 if x==director else 0)
df = df.drop(columns=['director_name'])

top_actors = []
for col in ['actor_1_name', 'actor_2_name', 'actor_3_name']:
    top_actors += list(df[col].value_counts().nlargest(10).index)
top_actors = list(set(top_actors))
for actor in top_actors:
    for col in ['actor_1_name', 'actor_2_name', 'actor_3_name']:
        df[f'actor_{actor}'] = df[col].apply(lambda x: 1 if x==actor else 0)
df = df.drop(columns=['actor_1_name','actor_2_name','actor_3_name'])

# ------------------------
# 6. Separate features and multi-target output
# ------------------------
X = df.drop(columns=['imdb_score','gross'])
y = df[['imdb_score','gross']].values  # Multi-target

# ------------------------
# 7. Scale numeric features
# ------------------------
scaler_X = StandardScaler()
X[numeric_cols[:-1]] = scaler_X.fit_transform(X[numeric_cols[:-1]])  # exclude gross from X

scaler_y = StandardScaler()
y = scaler_y.fit_transform(y)  # scale both imdb_score and gross

print("Preprocessing complete!")
print(f"Input shape: {X.shape}, Output shape: {y.shape}")


Preprocessing complete!
Input shape: (5043, 207), Output shape: (5043, 2)
